In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb

# Cargar el dataset
file_path = "../data/processed/train_data_processed_imputeKnn_scale.csv"
data = pd.read_csv(file_path)

# Inspección inicial
print("Dimensiones del dataset:", data.shape)
print(data.head())

# Identificar la variable objetivo y las características
target_col = "target"
features = [col for col in data.columns if col != target_col]

# Convertir todas las columnas categóricas a valores numéricos
label_encoders = {}
for col in features:
    if data[col].dtype == 'object':  # Verificar si la columna es categórica
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le

# Asegurarse de que 'therapeutic_area' esté presente
if 'therapeutic_area' not in data.columns:
    print("La columna 'therapeutic_area' no está presente en el dataset.")
else:
    # Filtrar por las diferentes áreas terapéuticas
    therapeutic_areas = data['therapeutic_area'].unique()
    print(f"Áreas terapéuticas encontradas: {therapeutic_areas}")

    # Parámetros para XGBoost
    params = {
        "objective": "reg:squarederror",
        "learning_rate": 0.05,
        "max_depth": 6,
        "min_child_weight": 1,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "reg_alpha": 0.1,
        "reg_lambda": 0.2,
        "seed": 42,
    }

    # GridSearchCV para optimizar los hiperparámetros de XGBoost
    param_grid_xgb = {
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0]
    }

    xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

    # Crear un diccionario para almacenar los resultados por cada área terapéutica
    therapeutic_area_rmse_scores = {}

    for area in therapeutic_areas:
        print(f"\nEntrenando modelo para el área terapéutica: {area}")
        
        # Filtrar los datos por área terapéutica
        area_data = data[data["therapeutic_area"] == area]
        X_area = area_data[features].values
        y_area = area_data[target_col].values

        # GridSearchCV para cada área terapéutica
        grid_search_xgb = GridSearchCV(xgb_model, param_grid_xgb, scoring=make_scorer(mean_squared_error, squared=False), cv=5, verbose=1)
        grid_search_xgb.fit(X_area, y_area)

        # Obtener el mejor modelo para esta área
        best_xgb_model = grid_search_xgb.best_estimator_
        print(f"Mejores parámetros de XGBoost para {area}: {grid_search_xgb.best_params_}")

        # Implementación de K-Fold Cross-Validation para este área terapéutica
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        rmse_scores = []

        for fold, (train_index, val_index) in enumerate(kf.split(X_area)):
            print(f"  Fold {fold + 1}")
            
            # Dividir los datos en conjuntos de entrenamiento y validación
            X_train, X_val = X_area[train_index], X_area[val_index]
            y_train, y_val = y_area[train_index], y_area[val_index]
            
            # Entrenar el modelo XGBoost con los mejores parámetros
            best_xgb_model.fit(X_train, y_train)
            
            # Predicción en el conjunto de validación
            y_pred = best_xgb_model.predict(X_val)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            rmse_scores.append(rmse)
            print(f"  Fold {fold + 1} RMSE: {rmse:.4f}")

        # Guardar el RMSE promedio para este área terapéutica
        therapeutic_area_rmse_scores[area] = np.mean(rmse_scores)
        print(f"RMSE promedio para el área terapéutica {area}: {therapeutic_area_rmse_scores[area]:.4f}")

    # Resumen general de los resultados
    print("\nResumen de RMSE por área terapéutica:")
    for area, rmse in therapeutic_area_rmse_scores.items():
        print(f"Área terapéutica {area}: RMSE promedio = {rmse:.4f}")


Dimensiones del dataset: (118917, 18)
        brand  che_pc_usd  che_perc_gdp corporation       country launch_date  \
0  BRAND_354E   -0.861595     -0.069532   CORP_D524  COUNTRY_88A3  2014-06-01   
1  BRAND_626D    1.012675      0.542469   CORP_01C7  COUNTRY_8B47  2014-06-01   
2  BRAND_45D9   -0.861595     -0.069532   CORP_39F7  COUNTRY_88A3  2014-06-01   
3  BRAND_D724    0.780386      1.333155   CORP_711A  COUNTRY_445D  2014-06-01   
4  BRAND_4887    0.626762      1.359908   CORP_443D  COUNTRY_D8B0  2014-06-01   

         date       drug_id                indication  insurance_perc_che  \
0  2014-06-01  DRUG_ID_8795              ['IND_C3B6']            1.150233   
1  2014-06-01  DRUG_ID_E66E  ['IND_1590', 'IND_ECAC']           -0.845488   
2  2014-06-01  DRUG_ID_F272              ['IND_B2EF']            1.150233   
3  2014-06-01  DRUG_ID_1D4E              ['IND_BAFB']           -1.332507   
4  2014-06-01  DRUG_ID_AA88              ['IND_3F31']            1.483735   

   populatio

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_reg

Mejores parámetros de XGBoost para 10: {'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 3, 'subsample': 1.0}
  Fold 1
  Fold 1 RMSE: 0.3713
  Fold 2
  Fold 2 RMSE: 0.3524
  Fold 3
  Fold 3 RMSE: 0.3943
  Fold 4
  Fold 4 RMSE: 0.3292
  Fold 5
  Fold 5 RMSE: 0.3332
RMSE promedio para el área terapéutica 10: 0.3561

Entrenando modelo para el área terapéutica: 9
Fitting 5 folds for each of 81 candidates, totalling 405 fits


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_reg

Mejores parámetros de XGBoost para 9: {'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 3, 'subsample': 0.8}
  Fold 1
  Fold 1 RMSE: 1.3783
  Fold 2
  Fold 2 RMSE: 1.3799
  Fold 3
  Fold 3 RMSE: 1.3040
  Fold 4
  Fold 4 RMSE: 1.2460
  Fold 5
  Fold 5 RMSE: 1.3163
RMSE promedio para el área terapéutica 9: 1.3249

Entrenando modelo para el área terapéutica: 7
Fitting 5 folds for each of 81 candidates, totalling 405 fits


...

Mejores parámetros de XGBoost para 3: {'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 3, 'subsample': 0.6}
  Fold 1
  Fold 1 RMSE: 0.1738
  Fold 2
  Fold 2 RMSE: 0.1646
  Fold 3
  Fold 3 RMSE: 0.1582
  Fold 4
  Fold 4 RMSE: 0.1700
  Fold 5
  Fold 5 RMSE: 0.1826
RMSE promedio para el área terapéutica 3: 0.1698

Entrenando modelo para el área terapéutica: 1
Fitting 5 folds for each of 81 candidates, totalling 405 fits


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_reg

Mejores parámetros de XGBoost para 1: {'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 3, 'subsample': 0.6}
  Fold 1
  Fold 1 RMSE: 0.2119
  Fold 2
  Fold 2 RMSE: 0.5408
  Fold 3
  Fold 3 RMSE: 0.3962
  Fold 4
  Fold 4 RMSE: 0.4445
  Fold 5
  Fold 5 RMSE: 0.2962
RMSE promedio para el área terapéutica 1: 0.3779

Entrenando modelo para el área terapéutica: 0
Fitting 5 folds for each of 81 candidates, totalling 405 fits


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_reg

Mejores parámetros de XGBoost para 0: {'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 3, 'subsample': 0.6}
  Fold 1
  Fold 1 RMSE: 0.3624
  Fold 2
  Fold 2 RMSE: 0.3475
  Fold 3
  Fold 3 RMSE: 0.3007
  Fold 4
  Fold 4 RMSE: 0.2900
  Fold 5
  Fold 5 RMSE: 0.3187
RMSE promedio para el área terapéutica 0: 0.3239

Resumen de RMSE por área terapéutica:
Área terapéutica 10: RMSE promedio = 0.3561
Área terapéutica 9: RMSE promedio = 1.3249
Área terapéutica 7: RMSE promedio = 0.6076
Área terapéutica 4: RMSE promedio = 0.4998
Área terapéutica 6: RMSE promedio = 0.5158
Área terapéutica 11: RMSE promedio = 1.0820
Área terapéutica 2: RMSE promedio = 0.0038
Área terapéutica 5: RMSE promedio = 0.0464
Área terapéutica 8: RMSE promedio = 0.1680
Área terapéutica 3: RMSE promedio = 0.1698
Área terapéutica 1: RMSE promedio = 0.3779
Área terapéutica 0: RMSE promedio = 0.3239
